# CFIA Importer
[![Static Badge](https://img.shields.io/badge/Jupyter_Notebook-F37726?style=for-the-badge)](https://jupyter.org/)

<br>

## Requirements
- Python (Version 3.6 or up)

<br>
<br>

### Install Required Dependencies
Run the code block below to install the required dependencies.

<br>

> ***📝 NOTE:*** <br>
>
> If you already have the required dependencies, you can *optionally* run the code block below.
> 
> (For this case, running the code will only import the required libraries without sending any HTTP server requests to Pypi)

In [1]:
###################
# Note: The code below is modified from AGRemap for dynamically installing
#   packages at runtime. That way, we do not need to make extra HTTP requests
#   to Pypi's server if the user's python environment already has the required libraries
#
# Reference:
#    https://github.com/nhok0169/Anime-Game-Remap/blob/nhok0169/Anime%20Game%20Remap%20(for%20all%20users)/api/src/FixRaidenBoss2/tools/PackageManager.py
#    https://github.com/nhok0169/Anime-Game-Remap/blob/nhok0169/Anime%20Game%20Remap%20(for%20all%20users)/api/src/FixRaidenBoss2/tools/PackageData.py


import pip._internal as pip
import importlib
from types import ModuleType
from typing import Optional, Dict


# PackageData: Class to hold data for importing a package
class PackageData():
    def __init__(self, module: str, installName: Optional[str] = None):
        self.module = module
        self.installName = module if (installName is None) else installName


# PackageManager: Class to manage the packages
class PackageManager():
    def __init__(self):
        self._packages: Dict[str, ModuleType] = {}

    # load(module, installName, save): Tries to import a package and install the package if the package
    #   is not installed yet. Can optionally save to cache.
    def load(self, module: str, installName: Optional[str] = None, save: bool = True) -> ModuleType:
        if (installName is None):
            installName = module

        try:
            return importlib.import_module(module)
        except ModuleNotFoundError:
            pip.main(['install', '-U', installName])

        result = importlib.import_module(module)
        if (save):
            self._packages[module] = result
        
        return result
    
    # get(packageData, cache): Retrieves a package and installs the package if the package is not installed yet.
    #   Has optional caching capability.
    def get(self, packageData: PackageData, cache: bool = True) -> ModuleType:
        if (not cache):
            return self.load(packageData.module, installName = packageData.installName, save = cache)

        result = None
        try:
            result = self._packages[packageData.module]
        except KeyError:
            result = self.load(packageData.module, installName = packageData.installName, save = cache)

        return result


##############################
# The required installations #
# ############################
Packages = [
    PackageData("pandas"),
    PackageData("openpyxl")
]

Packager = PackageManager()

for package in Packages:
    Packager.get(package, cache = False)


<br>

## User Settings

Below shows some configurations that may be different depending on the user.

<br>

> ***❇️ Important***
>
> Please ensure settings below are configured correctly.
>

In [2]:
import os

# The folder where the raw data files are located
DataFolder = "data"
CFIADataFolder = os.path.join(DataFolder, "CFIA")
HCDataFolder = os.path.join(DataFolder, "Health Canada")

# The file location to the output files 
OutputFolder = os.path.join("..", "..", "data")
CFIAOutputFileName = "CFIA Data"
HCOutputFileName = "Health Canada Data"


<br>

## Running the Importer

The code blocks below cleans up the raw CFIA data files to look similar to the Health Canada data

<br>

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import glob
import os
from threading import Lock, Thread
from enum import Enum
from typing import List, Dict, Callable


# Languages: Different languages available
class Languages(Enum):
    English = "en"
    French = "fr"


# CFIADataCols: Different columns for the CFIA data
class CFIADataCols(Enum):
    FoodGroup = "Food Group"
    FoodName = "Food Name"
    Agent = "Agent"
    Genus = "Genus"
    Species = "Species"
    SubSpecies = "Subspecies/ Genogroup"
    GenoType = "Genotype"
    SubGenoType = "Subgenotype"
    SeroType = "Serotype"
    OldOtherTyping = "Other typing"
    NewOtherTyping = "Other Typing"
    EColiCategory = "Ecoli CFIA Category"
    EColiTyped = "Ecoli Typed"
    IsVirulent = "EColi Virulent"
    Result = "Result"
    QualitativeResult = "Qualitative Result"
    QuantitativeResult = "Quantitative Result"
    QuantitativeResultOperator = "Quantitative Result Operator"
    QuantitativeResultUnit = "Quantitative Result Unit"
    ProductDescription = "Product Description"
    CountryOfOrigin = "Country of Origin"
    ResultComments = "Result Comments"
    SamplingLocationCityName = "Sampling Location City Name"
    ProjectCode = "Project Code"
    ProjectName = "Project Name"
    ProjectDescription = "Project Description"
    MethodComments = "Method Comments"
    MoreSampleInfo2 = "More Sample Info 2"
    SampleCode = "Sample Code"
    SampleCollectionDate = "Sample Collection Date"
    TestMethodCode = "Test Method Code"


# CFIAStrConsts: Some string keywords used in the raw CFIA data
class CFIAStrConsts(Enum):
    LangSeperator = "//"
    Virulent = "Virulent"
    NotVirulent = "Not Virulent"


# index position for each language part in to retrieve from the raw CFIA data
CFIALangPos = {Languages.English: 0, Languages.French: 1}

# Columns to have english and french seperated
CFIALangSeperatedCols = [CFIADataCols.FoodName.value, CFIADataCols.QuantitativeResultUnit.value, 
                         CFIADataCols.ProductDescription.value, CFIADataCols.CountryOfOrigin.value, 
                         CFIADataCols.ResultComments.value, CFIADataCols.SamplingLocationCityName.value,
                         CFIADataCols.ProjectCode.value, CFIADataCols.MethodComments.value,
                         CFIADataCols.SeroType.value]

# Columns that need translations
CFIANeedTranslationCols = [CFIADataCols.FoodGroup.value]

# Order for the columns in the output file
CFIAColOrder = [
    CFIADataCols.MoreSampleInfo2.value,
    CFIADataCols.FoodGroup.value,
    CFIADataCols.FoodName.value,
    CFIADataCols.SampleCollectionDate.value,
    CFIADataCols.SampleCode.value,
    CFIADataCols.SamplingLocationCityName.value,
    CFIADataCols.CountryOfOrigin.value,
    CFIADataCols.Agent.value,
    CFIADataCols.Genus.value,
    CFIADataCols.Species.value,
    CFIADataCols.SubSpecies.value,
    CFIADataCols.GenoType.value,
    CFIADataCols.SubGenoType.value,
    CFIADataCols.SeroType.value,
    CFIADataCols.EColiCategory.value,
    CFIADataCols.EColiTyped.value,
    CFIADataCols.IsVirulent.value,
    CFIADataCols.NewOtherTyping.value,
    CFIADataCols.TestMethodCode.value,
    CFIADataCols.MethodComments.value,
    CFIADataCols.Result.value,
    CFIADataCols.QualitativeResult.value,
    CFIADataCols.QuantitativeResultOperator.value,
    CFIADataCols.QuantitativeResult.value,
    CFIADataCols.QuantitativeResultUnit.value,
    CFIADataCols.ResultComments.value,
    CFIADataCols.ProjectCode.value,
    CFIADataCols.ProjectName.value,
    CFIADataCols.ProjectDescription.value,
    CFIADataCols.ProductDescription.value
]

CFIAStrCols = [
    CFIADataCols.Agent.value,
    CFIADataCols.Genus.value,
    CFIADataCols.Species.value,
    CFIADataCols.SubSpecies.value,
    CFIADataCols.GenoType.value,
    CFIADataCols.SubGenoType.value,
    CFIADataCols.SeroType.value,
    CFIADataCols.EColiCategory.value,
    CFIADataCols.EColiTyped.value,
    CFIADataCols.IsVirulent.value
]


# Translations for certain keywords
Translations = {
    Languages.English: {
        CFIADataCols.FoodGroup.value: {
            'Fish and fish products, including mollusks, crustaceans, and echinoderms': 'Fish and fish products, including mollusks, crustaceans, and echinoderms', 
            'Foodstuffs intended for particular nutritional uses': 'Foodstuffs intended for particular nutritional uses', 
            'Dairy products and analogues, excl butter ': 'Dairy products and analogues, excl butter ', 
            'Meat and meat products, including poultry and game': 'Meat and meat products, including poultry and game', 
            'Fruits and vegetables (incl fungi, legumes, aloe), seaweeds, nuts, seeds': 'Fruits and vegetables (incl fungi, legumes, aloe), seaweeds, nuts, seeds', 
            'Environmental Samples': 'Environmental Samples', 
            'Eggs and egg products': 'Eggs and egg products'
        },

        "yes": "yes",
        "no": "no",

        CFIADataCols.EColiCategory.value: {
            "O157": "O157", 
            "Verotoxigenic": "Verotoxigenic"
        },

        CFIADataCols.EColiTyped.value: {
            "Typed": "Typed",
            "Not Typed": "Not Typed"
        },

        CFIADataCols.QualitativeResult.value: {
            "Detected": "Detected",
            "Not Detected": "Not Detected",
            "Not Tested": "Not Tested"
        }
    },
    Languages.French: {
        CFIADataCols.FoodGroup.value: {
            'Fish and fish products, including mollusks, crustaceans, and echinoderms': 'Fish and fish products, including mollusks, crustaceans, and echinoderms', 
            'Foodstuffs intended for particular nutritional uses': 'Foodstuffs intended for particular nutritional uses', 
            'Dairy products and analogues, excl butter ': 'Dairy products and analogues, excl butter ', 
            'Meat and meat products, including poultry and game': 'Meat and meat products, including poultry and game', 
            'Fruits and vegetables (incl fungi, legumes, aloe), seaweeds, nuts, seeds': 'Fruits and vegetables (incl fungi, legumes, aloe), seaweeds, nuts, seeds', 
            'Environmental Samples': 'Environmental Samples', 
            'Eggs and egg products': 'Eggs and egg products'
        },

        "yes": "oui",
        "no": "non",

        CFIADataCols.EColiCategory.value: {
            "O157": "O157", 
            "Verotoxigenic": "Verotoxigenic"
        },

        CFIADataCols.EColiTyped.value: {
            "Typed": "Typed",
            "Not Typed": "Not Typed"
        },

        CFIADataCols.QualitativeResult.value: {
            "Detected": "Detected",
            "Not Detected": "Not Detected",
            "Not Tested": "Not Tested"
        }
    }
}


# ThreadManager: Class to manage running many threads
class ThreadManager():
    def __init__(self):
        self.threads = []

    # clear(): Clears all the threads
    def clear(self):
        self.threads.clear()

    # add(*args, **kwargs): Adds a thread
    def add(self, *args, **kwargs):
        self.threads.append(Thread(*args, **kwargs))

    # waitAll(): Runs all the threads at once and waits for all the threads to finish
    def waitAll(self):
        for thread in self.threads:
            thread.start()

        for thread in self.threads:
            thread.join()


# Importer: Base class for some data cleanup
class Importer():
    def __init__(self, dataFolder: str, outputFolder: str, outputFileName: str):
        self.dataFolder = dataFolder
        self.outputFolder = outputFolder
        self.outputFileName = outputFileName

        self._outputData = {}
        for lang in Languages:
            self._outputData[lang] = None

    # isEColi(rawData): Retrieves whether a certain row contains Bacteria EColi microorganism
    def isEColi(self, rawData: pd.DataFrame) -> pd.Series:
        agentVals = rawData[CFIADataCols.Agent.value]
        genusVals = rawData[CFIADataCols.Genus.value].fillna("")
        speciesVals = rawData[CFIADataCols.Species.value].fillna("")

        return ((agentVals == "Bacteria") & (genusVals == "Escherichia") & (speciesVals == "coli"))

    # run(): Runs the importer
    def run(self):
        pass


# Importer: Class to cleanup the CFIA data
class CFIAImporter(Importer):
    def __init__(self, dataFolder: str = CFIADataFolder, outputFolder: str = OutputFolder, outputFileName: str = CFIAOutputFileName):
        super().__init__(dataFolder, outputFolder, outputFileName)

        self._outputDataLocks = {}
        for lang in Languages:
            self._outputDataLocks[lang] = Lock()

    # _translateCol(col, rawData, data): Translates all the values in a single column
    def _translateCol(self, col: str, rawData: pd.DataFrame, data: Dict[Languages, pd.DataFrame]):
        colVals = rawData[col]

        for lang in Languages:
            translations = Translations[lang].get(col)
            if (translations is None):
                continue

            data[col] = colVals.apply(lambda colVal: translations.get(colVal, colVal))

    # _translate(rawData, data): Translates certain columns from the raw data
    def _translate(self, rawData: pd.DataFrame, data: Dict[Languages, pd.DataFrame]):
        for col in CFIANeedTranslationCols:
            self._translateCol(col, rawData, data)

    # _seperateTranslationCol(colValue): Seperate out a value consisting of the combined
    #   translations into individual translated parts
    @classmethod
    def _seperateTranslationVal(cls, colValue: str) -> List[str]:
        result = colValue.split(CFIAStrConsts.LangSeperator.value)
        result = list(map(lambda translatedPart: translatedPart.strip(), result))

        langLen = len(Languages)
        resultLen = len(result)

        if (resultLen == langLen):
            return result
        elif (resultLen > langLen):
            return result[:langLen]
        
        for i in range(resultLen, langLen):
            result.append(result[0])
        
        return result
    
    # _seperateTranslationCol(col, rawData, data): Seperate out the combined translated values in a column
    def _seperateTranslationCol(self, col: str, rawData: pd.DataFrame, data: Dict[Languages, pd.DataFrame]):
        colVals = rawData.get(col)
        if (colVals is None):
            return

        colVals = colVals.astype(str)[colVals.notnull()]
        colVals = colVals.apply(self._seperateTranslationVal)

        for lang in Languages:
            currentData = data[lang]
            langInd = CFIALangPos[lang]
            currentData[col] = colVals.apply(lambda langFoodGroups: langFoodGroups[langInd])

    # _seperateTranslations(rawData, data, rawDataLock): Seperate out the combined translated values from the raw data
    #   for each seperate language result data
    def _seperateTranslations(self, rawData: pd.DataFrame, data: Dict[Languages, pd.DataFrame]):
        for col in CFIALangSeperatedCols:
            self._seperateTranslationCol(col, rawData, data)

    # _createLangEColiCategory(lang, eColiBitArr, data): Creates the E-Coli Category for the data of a particular language
    def _createLangEColiCateogory(self, lang: Languages, eColiBitArr: List[pd.Series], data: Dict[Languages, pd.DataFrame]):
        currentData = data[lang]

        translations = Translations[lang][CFIADataCols.EColiCategory.value]
        eColiCategoryVals = np.select(eColiBitArr, [translations["O157"], translations["Verotoxigenic"]], default = "")

        currentData[CFIADataCols.EColiCategory.value] = eColiCategoryVals

    # _createLangEColiTyped(lang, eColiBitArr, data): Creates the E-Coli Typed column for the data of a particular language
    def _createLangEColiTyped(self, lang: Languages, eColiBitArr: List[pd.Series], data: Dict[Languages, pd.DataFrame]):
        currentData = data[lang]

        translations = Translations[lang][CFIADataCols.EColiTyped.value]
        eColiTypedVals = np.select(eColiBitArr, [translations["Typed"], translations["Not Typed"]], default = "")

        currentData[CFIADataCols.EColiTyped.value] = eColiTypedVals

    # _createEColiCategory(rawData, data, dataLocks): Creates the E-Coli category for the microorganism breadcrumb
    def _createEColiCategory(self, rawData: pd.DataFrame, data: Dict[Languages, pd.DataFrame]) -> np.array:
        seroTypeVals = rawData[CFIADataCols.SeroType.value].fillna("")
        otherTypingVals = rawData[CFIADataCols.NewOtherTyping.value].fillna("")

        isEColi = self.isEColi(rawData)

        seroTypeIsO157 = seroTypeVals.str.contains(pat = "O157", regex = False)
        seroTypeNotO157 = ~seroTypeIsO157
        eColiCateogryBitArr = [isEColi & seroTypeIsO157, isEColi & seroTypeNotO157]

        isNotTyped = (seroTypeVals == "") & (otherTypingVals == "")
        isTyped = ~isNotTyped
        eColiTypedBitArr = [isEColi & isTyped, isEColi & isNotTyped]

        for lang in Languages:
            self._createLangEColiCateogory(lang, eColiCateogryBitArr, data)
            self._createLangEColiTyped(lang, eColiTypedBitArr, data)

        return isTyped

    # _createLangIsVirulent(lang, virulentBitArr, data): 
    def _createLangIsVirulent(self, lang: Languages, virulentBitArr: List[pd.Series], data: Dict[Languages, pd.DataFrame]):
        currentData = data[lang]

        eColiTypedVals = np.select(virulentBitArr, [CFIAStrConsts.Virulent.value, CFIAStrConsts.NotVirulent.value], default = "")

        currentData[CFIADataCols.IsVirulent.value] = eColiTypedVals

    # _createEColiCategory(rawData, data, eColiTyped): Creates the isVirulent flag column for Bacteria EColi
    def _createIsVirulent(self, rawData: pd.DataFrame, data: Dict[Languages, pd.DataFrame], eColiTyped: np.array):
        seroTypeVals = rawData[CFIADataCols.SeroType.value].fillna("")
        seroTypeVals = seroTypeVals.str.replace(f"{CFIAStrConsts.LangSeperator.value}.*", "")

        commaPrefixedPatternStr = ".*,"
        isCommaPrefixed = seroTypeVals.str.match(commaPrefixedPatternStr)

        seroTypeVals = seroTypeVals.str.replace(commaPrefixedPatternStr, "", regex = True, n = 1)
        hasDetectedAfterComma = seroTypeVals.str.contains(pat = f"detected", case = False)

        isEColi = self.isEColi(rawData)

        isVirulent = isCommaPrefixed & ~hasDetectedAfterComma
        isNotVirulent = ~isVirulent

        virulentBitArr = [isEColi & eColiTyped & isVirulent, isEColi & eColiTyped & isNotVirulent]

        for lang in Languages:
            self._createLangIsVirulent(lang, virulentBitArr, data)

    def _readIndividual(self, key: str, combinedResult: Dict[str, pd.Series], readFunc: Callable[[], pd.Series], *args, **kwargs):
        result = readFunc(*args, **kwargs)
        combinedResult[key] = result

    # _createLangQualitative(lang, qualitativeVals, bitArr, data): Creates the qualitative results column for the data of a certain language
    def _createLangQualitative(self, lang: Languages, qualitativeVals: np.array, bitArr: List[pd.Series], data: Dict[Languages, pd.DataFrame]):
        currentData = data[lang]

        translations = Translations[lang][CFIADataCols.QualitativeResult.value]
        qualitativeVals = np.select(bitArr,[translations["Detected"], translations["Not Detected"], translations["Not Tested"]], default = translations["Not Detected"])

        currentData[CFIADataCols.QualitativeResult.value] = qualitativeVals

    # _createLangQuantitativeResult(lang, data): Creates the quantitative results column for the data of a certain language
    def _createLangQuantitativeResult(self, lang: Languages, bitArr: pd.Series, data: Dict[Languages, pd.DataFrame]):
        currentData = data[lang]
        resultVals = currentData[CFIADataCols.Result.value].astype(str)[currentData[CFIADataCols.Result.value].notnull()]

        quantitativeNums = resultVals.str.replace(">|>=|=|~|<|<=| |\t", "", case = False, regex = True)

        currentData[CFIADataCols.QuantitativeResult.value] = np.where(bitArr, quantitativeNums, "")

    # _createLangQuantitativeOperator(lang, bitArr, data): Creates the quantitative operator column for the data of a certain language
    def _createLangQuantitativeOperator(self, lang: Languages, bitArr: pd.Series, data: Dict[Languages, pd.DataFrame]):
        currentData = data[lang]
        resultVals = currentData[CFIADataCols.Result.value].astype(str)[currentData[CFIADataCols.Result.value].notnull()]

        quantitativeOps = resultVals.str.replace(r"[0-9]|-|\.| |\t", "", case = False, regex = True)
        isBlank = quantitativeOps == ""

        currentData[CFIADataCols.QuantitativeResultOperator.value] = np.select([bitArr & isBlank, bitArr & ~isBlank], ["=", quantitativeOps], default = "")

    # _createQualitativeAndQuantitative(rawData, data): Creates both the qualitative and quantitative results
    def _createQualitativeAndQuantitative(self, rawData: pd.DataFrame, data: Dict[Languages, pd.DataFrame]):
        rowCount = len(rawData.index)
        resultVals = rawData[CFIADataCols.Result.value].astype(str)[rawData[CFIADataCols.Result.value].notnull()]
        qualitative = np.zeros(rowCount)

        bitArrDict = {}

        self._readIndividual("isEmpty", bitArrDict, lambda col: col.isna(), resultVals)
        self._readIndividual("foundNotDetected", bitArrDict, lambda col: col.str.contains("Not Detected", case = False, na = False, regex = False), resultVals)
        self._readIndividual("foundNotAnalyzed", bitArrDict, lambda col: col.str.contains("Not Analyzed", case = False, na = False, regex = False), resultVals)
        self._readIndividual("foundDetected", bitArrDict, lambda col: col.str.match("^((?!Not).)*Detected", case = False, na = False), resultVals)
        self._readIndividual("foundPositive", bitArrDict, lambda col: col.str.match("^( |\t)*(((>|>=|=|~)( |\t)*-?[0-9]*(\.[0-9]+)?)|([1-9][0-9]*(\.[0-9]+)?))( |\t)*$", na = False), resultVals)
        self._readIndividual("foundNonPositive", bitArrDict, lambda col: col.str.match("^( |\t)*(((<|<=)( |\t)*-?[0-9]*(\.[0-9]+)?)|0+(\.[0-9]+)?|-[0-9]+(\.[0-9]+)?)( |\t)*$", na = False), resultVals)

        qualitativeBitArr = [bitArrDict["foundDetected"] | bitArrDict["foundPositive"], bitArrDict["foundNotDetected"] | bitArrDict["foundNonPositive"], bitArrDict["foundNotAnalyzed"]]
        quantitativeBitArr = bitArrDict["foundPositive"] | bitArrDict["foundNonPositive"]

        for lang in Languages:
            self._createLangQualitative(lang, qualitative, qualitativeBitArr, data)
            self._createLangQuantitativeResult(lang, quantitativeBitArr, data)
            self._createLangQuantitativeOperator(lang, quantitativeBitArr, data)

    # _mergeLangProcessedData(lang, processedData): Writes back the processed data from a single excel file to the merged result
    def _mergeLangProcessedData(self, lang: Languages, processedData: Dict[Languages, pd.DataFrame]):
        lock = self._outputDataLocks[lang]

        with lock:
            outputData = self._outputData[lang]
            self._outputData[lang] = processedData[lang].copy() if (outputData is None) else pd.concat([outputData, processedData[lang]])

    # processFile(excelFile): Cleanups a single excel file and merges the
    #   result into the output data
    def processFile(self, excelFile: str):
        rawData = pd.read_excel(excelFile)

        resultData = {}
        for lang in Languages:
            resultData[lang] = rawData.copy()

        rawData.rename(columns = {CFIADataCols.OldOtherTyping.value: CFIADataCols.NewOtherTyping.value}, inplace = True)

        self._translate(rawData, resultData)
        self._seperateTranslations(rawData, resultData)
        isEColiTyped = self._createEColiCategory(rawData, resultData)
        self._createQualitativeAndQuantitative(rawData, resultData)
        self._createIsVirulent(rawData, resultData, isEColiTyped)

        # write the processed data back to the combined output
        for lang in Languages:
            self._mergeLangProcessedData(lang, resultData)

    # _writeResult(lang): Writes back the processed result into the CSV file
    def _writeResult(self, lang: Languages):
        outputData = self._outputData[lang]
        if (outputData is None):
            return
        
        # reorder the columns
        outputData = outputData[CFIAColOrder]

        # convert the columns to string
        outputData = outputData.fillna('')
        outputData[CFIAStrCols] = outputData[CFIAStrCols].astype(str)[outputData.notnull()]

        file = os.path.join(self.outputFolder, f"{self.outputFileName}-{lang.value}.csv")
        print(f"Writing CFIA CSV output for language: {lang.value} ...\n", end = "")
        outputData.to_csv(file, index = False, encoding = "utf-8")

    def run(self):
        excelFiles = glob.glob(os.path.join(f"{self.dataFolder}", "*.xlsx"))

        threads = ThreadManager()
        for excelFile in excelFiles:
            threads.add(target = self.processFile, args=[excelFile], daemon=True)

        threads.waitAll()

        threads.clear()
        for lang in Languages:
            threads.add(target = self._writeResult, args = [lang])

        threads.waitAll()


class HCImporter(Importer):
    def __init__(self, dataFolder: str = HCDataFolder, outputFolder: str = OutputFolder, outputFileName: str = HCOutputFileName):
        super().__init__(dataFolder, outputFolder, outputFileName)

    def _cleanFile(self, lang: Languages, file: str):
        rawData = pd.read_csv(file)
        rawData.rename(columns = {CFIADataCols.OldOtherTyping.value: CFIADataCols.NewOtherTyping.value}, inplace = True)

        isEcoli = self.isEColi(rawData)
        notTypedStr = Translations[lang][CFIADataCols.EColiTyped.value]["Not Typed"]

        rawData[CFIADataCols.EColiTyped.value] = np.where(isEcoli, notTypedStr, "")
        rawData[CFIADataCols.IsVirulent.value] = ""

        outputFile = os.path.join(self.outputFolder, f"{self.outputFileName}-{lang.value}.csv")
        print(f"Writing HC CSV output for language: {lang.value} ...\n", end = "")
        rawData.to_csv(outputFile, index = False, encoding = "utf-8")

    def run(self):
        threads = ThreadManager()
        for lang in Languages:
            inputFile = os.path.join(self.dataFolder, f"CANLINE Micro w quant data - no protB values- export 2022-09-14-{lang.value}.csv")
            threads.add(target = self._cleanFile, args = [lang, inputFile])

        threads.waitAll()


########
# MAIN #
########
cfiaImporter = CFIAImporter()
hcImporter = HCImporter()

threads = ThreadManager()
threads.add(target = cfiaImporter.run)
threads.add(target = hcImporter.run)
threads.waitAll()

Writing HC CSV output for language: fr ...
Writing HC CSV output for language: en ...
Writing CFIA CSV output for language: fr ...
Writing CFIA CSV output for language: en ...
